In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize

# Paramètres globaux (réutilisables pour tous les pays)
risk_free_rate = 0.05 / 252  # Taux sans risque journalier (5% annuel)
source_start_date = "2000-02-01"
source_end_date = "2020-02-01"
target_start_date = "2015-02-01"
target_end_date = "2020-02-01"

## Royaume-Uni

In [63]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize
from scipy.stats import pearsonr
import random

# Global parameters
risk_free_rate = 0.05 / 252  # Daily risk-free rate (5% annual)
source_start_date = "2000-02-01"
source_end_date = "2020-02-01"
target_start_date = "2015-02-01"
target_end_date = "2020-02-01"
test_start_date = "2020-02-01"
test_end_date = "2021-09-01"
lambda_reg = 0.2  # Regularization parameter

# Fixed target assets (e.g., top 10 from FTSE 100 for UK)
target_assets = ['AZN.L', 'SHEL.L', 'HSBA.L', 'ULVR.L', 'BHP.L', 'REL.L', 'BATS.L', 'BP.L', 'LSEG.L', 'RR.L']

from datetime import datetime

# Function to get S&P 500 tickers with their addition date
def get_sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url)
    sp500_df = tables[0]

    # Convert the "Date added" column to datetime
    sp500_df['Date added'] = pd.to_datetime(sp500_df['Date added'], errors='coerce')

    # Filter tickers added after the year 2000
    sp500_df = sp500_df[sp500_df['Date added'] > datetime(2000, 1, 1)]

    return sp500_df["Symbol"].tolist()

# Function to randomly select n tickers added after 2000
def select_random_tickers(n=10):
    tickers = get_sp500_tickers()
    if not tickers:
        return "Impossible to retrieve tickers."
    return random.sample(tickers, n)


# Function to get S&P 500 tickers
def get_sp500_tickers2():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url)
    sp500_df = tables[0]
    return sp500_df["Symbol"].tolist()

# Function to randomly select n tickers
def select_random_tickers2(n=10):
    tickers = get_sp500_tickers()
    if not tickers:
        return "Impossible to retrieve tickers."
    return random.sample(tickers, n)



# Function to download excess returns
def download_excess_returns(tickers, start_date, end_date):
    data = {}
    for ticker in tickers:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        if not stock_data.empty:
            price_column = 'Adj Close' if 'Adj Close' in stock_data.columns else 'Close'
            stock_data['Excess Returns'] = stock_data[price_column].pct_change() - risk_free_rate
            data[ticker] = stock_data['Excess Returns']
    return pd.DataFrame(data).dropna()  # Drop rows with missing values


# Function to optimize the source portfolio (phi_S)
def optimize_source_portfolio(mu_S, Sigma_S):
    def sharpe_ratio_source(phi, mu_S, Sigma_S):
        numerator = np.dot(mu_S, phi)
        denominator = np.sqrt(np.dot(phi.T, np.dot(Sigma_S, phi)))
        return -numerator / denominator  # Negative because we minimize

    result_source = minimize(sharpe_ratio_source, np.ones(len(mu_S)) / len(mu_S), args=(mu_S, Sigma_S),
                             method='SLSQP', bounds=[(0, 1) for _ in range(len(mu_S))],
                             constraints=[{'type': 'eq', 'fun': lambda phi: np.sum(phi) - 1}])
    return result_source.x

def optimize_source_portfolio2(mu_S, Sigma_S):
    def sharpe_ratio_source(phi, mu_S, Sigma_S):
        numerator = np.dot(mu_S, phi)
        denominator = np.sqrt(np.dot(phi.T, np.dot(Sigma_S, phi)))
        return -numerator / denominator  # Negative because we minimize

    result_source = minimize(sharpe_ratio_source, np.ones(len(mu_S)) / len(mu_S), args=(mu_S, Sigma_S),
                             method='SLSQP', bounds=[(0, 1) for _ in range(len(mu_S))],
                             constraints=[{'type': 'eq', 'fun': lambda phi: np.sum(phi) - 1}])
    return -result_source.fun

# Function to optimize the target portfolio with regularization (phi_T)
def optimize_target_portfolio(mu_T, Sigma_T, phi_S, lambda_reg):
    def regularized_sharpe_ratio(phi, mu_T, Sigma_T, phi_S, lambda_reg):
        numerator = np.dot(mu_T, phi)
        denominator = np.sqrt(np.dot(phi.T, np.dot(Sigma_T, phi)))
        sharpe_ratio = numerator / denominator
        regularization = lambda_reg * np.linalg.norm(phi_S - phi) ** 2  # Regularization term
        return -(sharpe_ratio - regularization)  # Negative because we minimize

    result_target = minimize(regularized_sharpe_ratio, np.ones(len(mu_T)) / len(mu_T),
                             args=(mu_T, Sigma_T, phi_S, lambda_reg), method='SLSQP',
                             bounds=[(0, 1) for _ in range(len(mu_T))],
                             constraints=[{'type': 'eq', 'fun': lambda phi: np.sum(phi) - 1}])
    return result_target.x, -result_target.fun

def optimize_target_portfolio2(mu_T, Sigma_T, phi_S, lambda_reg):
    def regularized_sharpe_ratio(phi, mu_T, Sigma_T, phi_S, lambda_reg):
        numerator = np.dot(mu_T, phi)
        denominator = np.sqrt(np.dot(phi.T, np.dot(Sigma_T, phi)))
        sharpe_ratio = numerator / denominator
        regularization = lambda_reg * np.linalg.norm(phi_S - phi) ** 2  # Regularization term
        return -(sharpe_ratio - regularization)  # Negative because we minimize

    result_target = minimize(regularized_sharpe_ratio, np.ones(len(mu_T)) / len(mu_T),
                             args=(mu_T, Sigma_T, phi_S, lambda_reg), method='SLSQP',
                             bounds=[(0, 1) for _ in range(len(mu_T))],
                             constraints=[{'type': 'eq', 'fun': lambda phi: np.sum(phi) - 1}])
    return -result_target.fun

# Function to compute transfer risk
# Function to compute R1 (Transfer Risk)
def compute_R1(phi, mu_T, Sigma_T):
    numerator = np.dot(mu_T, phi)
    denominator = np.sqrt(np.dot(phi.T, np.dot(Sigma_T, phi)))
    ratio = numerator / denominator
    R1 = 1.0 / ratio
    return R1

def compute_transfer_risk(phi_T, source_data, target_test_data):
    mu_S = np.array(source_data.mean())
    Sigma_S = np.array(source_data.cov())
    mu_T_test = np.array(target_test_data.mean())
    Sigma_T_test = np.array(target_test_data.cov())

    # Compute R1 (inverse of source portfolio Sharpe ratio)
    R1 = compute_R1(optimize_source_portfolio(mu_S, Sigma_S), mu_S, Sigma_S)

    # Compute R2 (Wasserstein-2 distance between source and target distributions)
    R2 = np.linalg.norm(mu_S - mu_T_test) + np.linalg.norm(Sigma_S - Sigma_T_test, ord=2)
    return R1 + R2



# Main experiment loop
n_experiments = 10  # Reduced to 100 for faster testing; increase to 1000 for final results
sharpe_ratios_transfer = []
sharpe_ratios_direct = []
transfer_risks = []

# Download target data
target_data = download_excess_returns(target_assets, target_start_date, test_end_date)
target_train_data = target_data.loc[:target_end_date]  # Training data up to February 2020
target_test_data = target_data.loc[test_start_date:]   # Testing data from February 2020 onwards

# Compute target mean and covariance for training data
mu_T = np.array(target_train_data.mean())
Sigma_T = np.array(target_train_data.cov())

for _ in range(n_experiments):
    try:
        # Randomly select source assets (10 random S&P 500 stocks)
        source_assets = select_random_tickers(n=10)
        print("Source Assets:", source_assets)

        # Generate source data
        source_data = download_excess_returns(source_assets, source_start_date, source_end_date)
        if not isinstance(source_data, pd.DataFrame) or source_data.shape[1] != 10:
            print("Skipping experiment due to missing data for some source assets.")
            continue

        # Optimize source portfolio (phi_S)
        mu_S = np.array(source_data.mean())
        Sigma_S = np.array(source_data.cov())
        phi_S = optimize_source_portfolio(mu_S, Sigma_S)

        # Optimize target portfolio with transfer learning (phi_T)
        phi_T, sharpe_ratio_transfer = optimize_target_portfolio(mu_T, Sigma_T, phi_S, lambda_reg)

        # Optimize target portfolio with direct learning (phi_T_direct)
        phi_T_direct = optimize_source_portfolio(mu_T, Sigma_T)

        # Compute target mean and covariance for testing data
        mu_T_test = np.array(target_test_data.mean())
        Sigma_T_test = np.array(target_test_data.cov())

        # Compute Sharpe ratios on testing data (inverser compute_R1 pour obtenir le vrai ratio)
        #sharpe_ratio_transfer_test = 1.0 / compute_R1(phi_S, mu_T_test, Sigma_T_test)
        #sharpe_ratio_direct_test = 1.0 / compute_R1(phi_T_direct, mu_T_test, Sigma_T_test)
        #phiTwhatever, sharpe_ratio_transfer_test = optimize_target_portfolio(mu_T_test, Sigma_T_test, phi_S, lambda_reg)
        sharpe_ratio_transfer_test = optimize_target_portfolio2(mu_T_test, Sigma_T_test, phi_S, lambda_reg)
        sharpe_ratio_direct_test= optimize_source_portfolio2(mu_T_test, Sigma_T_test)

        # Compute transfer risk
        R1 = compute_R1(phi_S, mu_S, Sigma_S)  # Correction : calculer R1 avec phi_S
        R2 = np.linalg.norm(mu_S - mu_T_test) + np.linalg.norm(Sigma_S - Sigma_T_test, ord=2)
        transfer_risk = R1 + R2

        # Store results
        sharpe_ratios_transfer.append(sharpe_ratio_transfer)
        sharpe_ratios_direct.append(sharpe_ratio_direct_test)
        transfer_risks.append(transfer_risk)

    except Exception as e:
        print(f"Error in experiment {_}: {e}")
        continue


# Compute correlation between Sharpe ratios and transfer risks
if len(sharpe_ratios_transfer) > 1 and len(transfer_risks) > 1:
    correlation, p_value = pearsonr(sharpe_ratios_transfer, transfer_risks)
    print("Correlation between Sharpe Ratio (Transfer) and Transfer Risk:", correlation)
    print("p-value:", p_value)
else:
    print("Not enough experiments completed to compute correlation.")
    print(transfer_risks)
    print(sharpe_ratios_transfer)

    print(sharpe_ratios_direct)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Source Assets: ['POOL', 'DD', 'NCLH', 'TXN', 'AME', 'IDXX', 'IT', 'LULU', 'AKAM', 'BXP']


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Source Assets: ['MCHP', 'MAA', 'RCL', 'CHTR', 'WEC', 'FANG', 'CE', 'LYB', 'WELL', 'MHK']


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

Skipping experiment due to missing data for some source assets.
Source Assets: ['CTRA', 'CRWD', 'WRB', 'ULTA', 'LW', 'KDP', 'FAST', 'FOXA', 'CRL', 'FRT']


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['PLTR']: YFPricesMissingError('possibly delisted; no price data found  (1d 2000-02-01 -> 2020-02-01) (Yahoo error = "Data doesn\'t exist for startDate = 949381200, endDate = 1580533200")')


Source Assets: ['PLTR', 'CFG', 'DD', 'RSG', 'POOL', 'KVUE', 'CE', 'ELV', 'MTCH', 'SBUX']


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['KVUE']: YFPricesMissingError('possibly delisted; no price data found  (1d 2000-02-01 -> 2020-02-01) (Yahoo error = "Data doesn\'t exist for startDate = 949381200, endDate = 1580533200")')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Skipping experiment due to missing data for some source assets.
Correlation between Sharpe Ratio (Transfer) and Transfer Risk: 0.6700660013005417
p-value: 0.06903647951625742


## Brésil

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize
from scipy.stats import pearsonr
import random

# Global parameters
risk_free_rate = 0.05 / 252  # Daily risk-free rate (5% annual)
source_start_date = "2000-02-01"
source_end_date = "2020-02-01"
target_start_date = "2015-02-01"
target_end_date = "2020-02-01"
test_start_date = "2020-02-01"
test_end_date = "2021-09-01"
lambda_reg = 0.2  # Regularization parameter

# Fixed target assets (e.g., top 10 from BOVESPA)
target_assets= ['PETR4.SA', 'VALE3.SA', 'ITUB4.SA', 'BBDC4.SA', 'ABEV3.SA',
                'BBAS3.SA', 'B3SA3.SA', 'WEGE3.SA', 'EQTL3.SA', 'SUZB3.SA']

from datetime import datetime

# Function to get S&P 500 tickers with their addition date
def get_sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url)
    sp500_df = tables[0]

    # Convert the "Date added" column to datetime
    sp500_df['Date added'] = pd.to_datetime(sp500_df['Date added'], errors='coerce')

    # Filter tickers added after the year 2000
    sp500_df = sp500_df[sp500_df['Date added'] > datetime(2000, 1, 1)]

    return sp500_df["Symbol"].tolist()

# Function to randomly select n tickers added after 2000
def select_random_tickers(n=10):
    tickers = get_sp500_tickers()
    if not tickers:
        return "Impossible to retrieve tickers."
    return random.sample(tickers, n)


# Function to get S&P 500 tickers
def get_sp500_tickers2():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url)
    sp500_df = tables[0]
    return sp500_df["Symbol"].tolist()

# Function to randomly select n tickers
def select_random_tickers2(n=10):
    tickers = get_sp500_tickers()
    if not tickers:
        return "Impossible to retrieve tickers."
    return random.sample(tickers, n)



# Function to download excess returns
def download_excess_returns(tickers, start_date, end_date):
    data = {}
    for ticker in tickers:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        if not stock_data.empty:
            price_column = 'Adj Close' if 'Adj Close' in stock_data.columns else 'Close'
            stock_data['Excess Returns'] = stock_data[price_column].pct_change() - risk_free_rate
            data[ticker] = stock_data['Excess Returns']
    return pd.DataFrame(data).dropna()  # Drop rows with missing values


# Function to optimize the source portfolio (phi_S)
def optimize_source_portfolio(mu_S, Sigma_S):
    def sharpe_ratio_source(phi, mu_S, Sigma_S):
        numerator = np.dot(mu_S, phi)
        denominator = np.sqrt(np.dot(phi.T, np.dot(Sigma_S, phi)))
        return -numerator / denominator  # Negative because we minimize

    result_source = minimize(sharpe_ratio_source, np.ones(len(mu_S)) / len(mu_S), args=(mu_S, Sigma_S),
                             method='SLSQP', bounds=[(0, 1) for _ in range(len(mu_S))],
                             constraints=[{'type': 'eq', 'fun': lambda phi: np.sum(phi) - 1}])
    return result_source.x

def optimize_source_portfolio2(mu_S, Sigma_S):
    def sharpe_ratio_source(phi, mu_S, Sigma_S):
        numerator = np.dot(mu_S, phi)
        denominator = np.sqrt(np.dot(phi.T, np.dot(Sigma_S, phi)))
        return -numerator / denominator  # Negative because we minimize

    result_source = minimize(sharpe_ratio_source, np.ones(len(mu_S)) / len(mu_S), args=(mu_S, Sigma_S),
                             method='SLSQP', bounds=[(0, 1) for _ in range(len(mu_S))],
                             constraints=[{'type': 'eq', 'fun': lambda phi: np.sum(phi) - 1}])
    return -result_source.fun

# Function to optimize the target portfolio with regularization (phi_T)
def optimize_target_portfolio(mu_T, Sigma_T, phi_S, lambda_reg):
    def regularized_sharpe_ratio(phi, mu_T, Sigma_T, phi_S, lambda_reg):
        numerator = np.dot(mu_T, phi)
        denominator = np.sqrt(np.dot(phi.T, np.dot(Sigma_T, phi)))
        sharpe_ratio = numerator / denominator
        regularization = lambda_reg * np.linalg.norm(phi_S - phi) ** 2  # Regularization term
        return -(sharpe_ratio - regularization)  # Negative because we minimize

    result_target = minimize(regularized_sharpe_ratio, np.ones(len(mu_T)) / len(mu_T),
                             args=(mu_T, Sigma_T, phi_S, lambda_reg), method='SLSQP',
                             bounds=[(0, 1) for _ in range(len(mu_T))],
                             constraints=[{'type': 'eq', 'fun': lambda phi: np.sum(phi) - 1}])
    return result_target.x, -result_target.fun

def optimize_target_portfolio2(mu_T, Sigma_T, phi_S, lambda_reg):
    def regularized_sharpe_ratio(phi, mu_T, Sigma_T, phi_S, lambda_reg):
        numerator = np.dot(mu_T, phi)
        denominator = np.sqrt(np.dot(phi.T, np.dot(Sigma_T, phi)))
        sharpe_ratio = numerator / denominator
        regularization = lambda_reg * np.linalg.norm(phi_S - phi) ** 2  # Regularization term
        return -(sharpe_ratio - regularization)  # Negative because we minimize

    result_target = minimize(regularized_sharpe_ratio, np.ones(len(mu_T)) / len(mu_T),
                             args=(mu_T, Sigma_T, phi_S, lambda_reg), method='SLSQP',
                             bounds=[(0, 1) for _ in range(len(mu_T))],
                             constraints=[{'type': 'eq', 'fun': lambda phi: np.sum(phi) - 1}])
    return -result_target.fun

# Function to compute transfer risk
# Function to compute R1 (Transfer Risk)
def compute_R1(phi, mu_T, Sigma_T):
    numerator = np.dot(mu_T, phi)
    denominator = np.sqrt(np.dot(phi.T, np.dot(Sigma_T, phi)))
    ratio = numerator / denominator
    R1 = 1.0 / ratio
    return R1

def compute_transfer_risk(phi_T, source_data, target_test_data):
    mu_S = np.array(source_data.mean())
    Sigma_S = np.array(source_data.cov())
    mu_T_test = np.array(target_test_data.mean())
    Sigma_T_test = np.array(target_test_data.cov())

    # Compute R1 (inverse of source portfolio Sharpe ratio)
    R1 = compute_R1(optimize_source_portfolio(mu_S, Sigma_S), mu_S, Sigma_S)

    # Compute R2 (Wasserstein-2 distance between source and target distributions)
    R2 = np.linalg.norm(mu_S - mu_T_test) + np.linalg.norm(Sigma_S - Sigma_T_test, ord=2)
    return R1 + R2



# Main experiment loop
n_experiments = 10  # Reduced to 100 for faster testing; increase to 1000 for final results
sharpe_ratios_transfer = []
sharpe_ratios_direct = []
transfer_risks = []

# Download target data
target_data = download_excess_returns(target_assets, target_start_date, test_end_date)
target_train_data = target_data.loc[:target_end_date]  # Training data up to February 2020
target_test_data = target_data.loc[test_start_date:]   # Testing data from February 2020 onwards

# Compute target mean and covariance for training data
mu_T = np.array(target_train_data.mean())
Sigma_T = np.array(target_train_data.cov())

for _ in range(n_experiments):
    try:
        # Randomly select source assets (10 random S&P 500 stocks)
        source_assets = select_random_tickers(n=10)
        print("Source Assets:", source_assets)

        # Generate source data
        source_data = download_excess_returns(source_assets, source_start_date, source_end_date)
        if not isinstance(source_data, pd.DataFrame) or source_data.shape[1] != 10:
            print("Skipping experiment due to missing data for some source assets.")
            continue

        # Optimize source portfolio (phi_S)
        mu_S = np.array(source_data.mean())
        Sigma_S = np.array(source_data.cov())
        phi_S = optimize_source_portfolio(mu_S, Sigma_S)

        # Optimize target portfolio with transfer learning (phi_T)
        phi_T, sharpe_ratio_transfer = optimize_target_portfolio(mu_T, Sigma_T, phi_S, lambda_reg)

        # Optimize target portfolio with direct learning (phi_T_direct)
        phi_T_direct = optimize_source_portfolio(mu_T, Sigma_T)

        # Compute target mean and covariance for testing data
        mu_T_test = np.array(target_test_data.mean())
        Sigma_T_test = np.array(target_test_data.cov())

        # Compute Sharpe ratios on testing data (inverser compute_R1 pour obtenir le vrai ratio)
        #sharpe_ratio_transfer_test = 1.0 / compute_R1(phi_S, mu_T_test, Sigma_T_test)
        #sharpe_ratio_direct_test = 1.0 / compute_R1(phi_T_direct, mu_T_test, Sigma_T_test)
        #phiTwhatever, sharpe_ratio_transfer_test = optimize_target_portfolio(mu_T_test, Sigma_T_test, phi_S, lambda_reg)
        sharpe_ratio_transfer_test = optimize_target_portfolio2(mu_T_test, Sigma_T_test, phi_S, lambda_reg)
        sharpe_ratio_direct_test= optimize_source_portfolio2(mu_T_test, Sigma_T_test)

        # Compute transfer risk
        R1 = compute_R1(phi_S, mu_S, Sigma_S)  # Correction : calculer R1 avec phi_S
        R2 = np.linalg.norm(mu_S - mu_T_test) + np.linalg.norm(Sigma_S - Sigma_T_test, ord=2)
        transfer_risk = R1 + R2

        # Store results
        sharpe_ratios_transfer.append(sharpe_ratio_transfer)
        sharpe_ratios_direct.append(sharpe_ratio_direct_test)
        transfer_risks.append(transfer_risk)

    except Exception as e:
        print(f"Error in experiment {_}: {e}")
        continue


# Compute correlation between Sharpe ratios and transfer risks
if len(sharpe_ratios_transfer) > 1 and len(transfer_risks) > 1:
    correlation, p_value = pearsonr(sharpe_ratios_transfer, transfer_risks)
    print("Correlation between Sharpe Ratio (Transfer) and Transfer Risk:", correlation)
    print("p-value:", p_value)
else:
    print("Not enough experiments completed to compute correlation.")
    print(transfer_risks)
    print(sharpe_ratios_transfer)

    print(sharpe_ratios_direct)

# Allemagne

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize
from scipy.stats import pearsonr
import random

# Global parameters
risk_free_rate = 0.05 / 252  # Daily risk-free rate (5% annual)
source_start_date = "2000-02-01"
source_end_date = "2020-02-01"
target_start_date = "2015-02-01"
target_end_date = "2020-02-01"
test_start_date = "2020-02-01"
test_end_date = "2021-09-01"
lambda_reg = 0.2  # Regularization parameter

# Fixed target assets (e.g., top 10 from DAX)

target_assets= [
    'SAP.DE', 'SIE.DE', 'DTE.DE', 'ALV.DE', 'MUV2.DE', 'MRK.DE', 'SHL.DE', 'BMW.DE', 'MBG.DE', 'VOW.DE']

from datetime import datetime

# Function to get S&P 500 tickers with their addition date
def get_sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url)
    sp500_df = tables[0]

    # Convert the "Date added" column to datetime
    sp500_df['Date added'] = pd.to_datetime(sp500_df['Date added'], errors='coerce')

    # Filter tickers added after the year 2000
    sp500_df = sp500_df[sp500_df['Date added'] > datetime(2000, 1, 1)]

    return sp500_df["Symbol"].tolist()

# Function to randomly select n tickers added after 2000
def select_random_tickers(n=10):
    tickers = get_sp500_tickers()
    if not tickers:
        return "Impossible to retrieve tickers."
    return random.sample(tickers, n)


# Function to get S&P 500 tickers
def get_sp500_tickers2():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url)
    sp500_df = tables[0]
    return sp500_df["Symbol"].tolist()

# Function to randomly select n tickers
def select_random_tickers2(n=10):
    tickers = get_sp500_tickers()
    if not tickers:
        return "Impossible to retrieve tickers."
    return random.sample(tickers, n)



# Function to download excess returns
def download_excess_returns(tickers, start_date, end_date):
    data = {}
    for ticker in tickers:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        if not stock_data.empty:
            price_column = 'Adj Close' if 'Adj Close' in stock_data.columns else 'Close'
            stock_data['Excess Returns'] = stock_data[price_column].pct_change() - risk_free_rate
            data[ticker] = stock_data['Excess Returns']
    return pd.DataFrame(data).dropna()  # Drop rows with missing values


# Function to optimize the source portfolio (phi_S)
def optimize_source_portfolio(mu_S, Sigma_S):
    def sharpe_ratio_source(phi, mu_S, Sigma_S):
        numerator = np.dot(mu_S, phi)
        denominator = np.sqrt(np.dot(phi.T, np.dot(Sigma_S, phi)))
        return -numerator / denominator  # Negative because we minimize

    result_source = minimize(sharpe_ratio_source, np.ones(len(mu_S)) / len(mu_S), args=(mu_S, Sigma_S),
                             method='SLSQP', bounds=[(0, 1) for _ in range(len(mu_S))],
                             constraints=[{'type': 'eq', 'fun': lambda phi: np.sum(phi) - 1}])
    return result_source.x

def optimize_source_portfolio2(mu_S, Sigma_S):
    def sharpe_ratio_source(phi, mu_S, Sigma_S):
        numerator = np.dot(mu_S, phi)
        denominator = np.sqrt(np.dot(phi.T, np.dot(Sigma_S, phi)))
        return -numerator / denominator  # Negative because we minimize

    result_source = minimize(sharpe_ratio_source, np.ones(len(mu_S)) / len(mu_S), args=(mu_S, Sigma_S),
                             method='SLSQP', bounds=[(0, 1) for _ in range(len(mu_S))],
                             constraints=[{'type': 'eq', 'fun': lambda phi: np.sum(phi) - 1}])
    return -result_source.fun

# Function to optimize the target portfolio with regularization (phi_T)
def optimize_target_portfolio(mu_T, Sigma_T, phi_S, lambda_reg):
    def regularized_sharpe_ratio(phi, mu_T, Sigma_T, phi_S, lambda_reg):
        numerator = np.dot(mu_T, phi)
        denominator = np.sqrt(np.dot(phi.T, np.dot(Sigma_T, phi)))
        sharpe_ratio = numerator / denominator
        regularization = lambda_reg * np.linalg.norm(phi_S - phi) ** 2  # Regularization term
        return -(sharpe_ratio - regularization)  # Negative because we minimize

    result_target = minimize(regularized_sharpe_ratio, np.ones(len(mu_T)) / len(mu_T),
                             args=(mu_T, Sigma_T, phi_S, lambda_reg), method='SLSQP',
                             bounds=[(0, 1) for _ in range(len(mu_T))],
                             constraints=[{'type': 'eq', 'fun': lambda phi: np.sum(phi) - 1}])
    return result_target.x, -result_target.fun

def optimize_target_portfolio2(mu_T, Sigma_T, phi_S, lambda_reg):
    def regularized_sharpe_ratio(phi, mu_T, Sigma_T, phi_S, lambda_reg):
        numerator = np.dot(mu_T, phi)
        denominator = np.sqrt(np.dot(phi.T, np.dot(Sigma_T, phi)))
        sharpe_ratio = numerator / denominator
        regularization = lambda_reg * np.linalg.norm(phi_S - phi) ** 2  # Regularization term
        return -(sharpe_ratio - regularization)  # Negative because we minimize

    result_target = minimize(regularized_sharpe_ratio, np.ones(len(mu_T)) / len(mu_T),
                             args=(mu_T, Sigma_T, phi_S, lambda_reg), method='SLSQP',
                             bounds=[(0, 1) for _ in range(len(mu_T))],
                             constraints=[{'type': 'eq', 'fun': lambda phi: np.sum(phi) - 1}])
    return -result_target.fun

# Function to compute transfer risk
# Function to compute R1 (Transfer Risk)
def compute_R1(phi, mu_T, Sigma_T):
    numerator = np.dot(mu_T, phi)
    denominator = np.sqrt(np.dot(phi.T, np.dot(Sigma_T, phi)))
    ratio = numerator / denominator
    R1 = 1.0 / ratio
    return R1

def compute_transfer_risk(phi_T, source_data, target_test_data):
    mu_S = np.array(source_data.mean())
    Sigma_S = np.array(source_data.cov())
    mu_T_test = np.array(target_test_data.mean())
    Sigma_T_test = np.array(target_test_data.cov())

    # Compute R1 (inverse of source portfolio Sharpe ratio)
    R1 = compute_R1(optimize_source_portfolio(mu_S, Sigma_S), mu_S, Sigma_S)

    # Compute R2 (Wasserstein-2 distance between source and target distributions)
    R2 = np.linalg.norm(mu_S - mu_T_test) + np.linalg.norm(Sigma_S - Sigma_T_test, ord=2)
    return R1 + R2



# Main experiment loop
n_experiments = 10  # Reduced to 100 for faster testing; increase to 1000 for final results
sharpe_ratios_transfer = []
sharpe_ratios_direct = []
transfer_risks = []

# Download target data
target_data = download_excess_returns(target_assets, target_start_date, test_end_date)
target_train_data = target_data.loc[:target_end_date]  # Training data up to February 2020
target_test_data = target_data.loc[test_start_date:]   # Testing data from February 2020 onwards

# Compute target mean and covariance for training data
mu_T = np.array(target_train_data.mean())
Sigma_T = np.array(target_train_data.cov())

for _ in range(n_experiments):
    try:
        # Randomly select source assets (10 random S&P 500 stocks)
        source_assets = select_random_tickers(n=10)
        print("Source Assets:", source_assets)

        # Generate source data
        source_data = download_excess_returns(source_assets, source_start_date, source_end_date)
        if not isinstance(source_data, pd.DataFrame) or source_data.shape[1] != 10:
            print("Skipping experiment due to missing data for some source assets.")
            continue

        # Optimize source portfolio (phi_S)
        mu_S = np.array(source_data.mean())
        Sigma_S = np.array(source_data.cov())
        phi_S = optimize_source_portfolio(mu_S, Sigma_S)

        # Optimize target portfolio with transfer learning (phi_T)
        phi_T, sharpe_ratio_transfer = optimize_target_portfolio(mu_T, Sigma_T, phi_S, lambda_reg)

        # Optimize target portfolio with direct learning (phi_T_direct)
        phi_T_direct = optimize_source_portfolio(mu_T, Sigma_T)

        # Compute target mean and covariance for testing data
        mu_T_test = np.array(target_test_data.mean())
        Sigma_T_test = np.array(target_test_data.cov())

        # Compute Sharpe ratios on testing data (inverser compute_R1 pour obtenir le vrai ratio)
        #sharpe_ratio_transfer_test = 1.0 / compute_R1(phi_S, mu_T_test, Sigma_T_test)
        #sharpe_ratio_direct_test = 1.0 / compute_R1(phi_T_direct, mu_T_test, Sigma_T_test)
        #phiTwhatever, sharpe_ratio_transfer_test = optimize_target_portfolio(mu_T_test, Sigma_T_test, phi_S, lambda_reg)
        sharpe_ratio_transfer_test = optimize_target_portfolio2(mu_T_test, Sigma_T_test, phi_S, lambda_reg)
        sharpe_ratio_direct_test= optimize_source_portfolio2(mu_T_test, Sigma_T_test)

        # Compute transfer risk
        R1 = compute_R1(phi_S, mu_S, Sigma_S)  # Correction : calculer R1 avec phi_S
        R2 = np.linalg.norm(mu_S - mu_T_test) + np.linalg.norm(Sigma_S - Sigma_T_test, ord=2)
        transfer_risk = R1 + R2

        # Store results
        sharpe_ratios_transfer.append(sharpe_ratio_transfer)
        sharpe_ratios_direct.append(sharpe_ratio_direct_test)
        transfer_risks.append(transfer_risk)

    except Exception as e:
        print(f"Error in experiment {_}: {e}")
        continue


# Compute correlation between Sharpe ratios and transfer risks
if len(sharpe_ratios_transfer) > 1 and len(transfer_risks) > 1:
    correlation, p_value = pearsonr(sharpe_ratios_transfer, transfer_risks)
    print("Correlation between Sharpe Ratio (Transfer) and Transfer Risk:", correlation)
    print("p-value:", p_value)
else:
    print("Not enough experiments completed to compute correlation.")
    print(transfer_risks)
    print(sharpe_ratios_transfer)

    print(sharpe_ratios_direct)

# Singapour

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize
from scipy.stats import pearsonr
import random

# Global parameters
risk_free_rate = 0.05 / 252  # Daily risk-free rate (5% annual)
source_start_date = "2000-02-01"
source_end_date = "2020-02-01"
target_start_date = "2015-02-01"
target_end_date = "2020-02-01"
test_start_date = "2020-02-01"
test_end_date = "2021-09-01"
lambda_reg = 0.2  # Regularization parameter

# Fixed target assets (e.g., top 10 from SGX)

target_assets= [
    'D05.SI', 'O39.SI', 'U11.SI', 'Z74.SI', 'F34.SI', 'C6L.SI', 'J36.SI', 'S63.SI', 'C38U.SI', 'Y92.SI']

from datetime import datetime

# Function to get S&P 500 tickers with their addition date
def get_sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url)
    sp500_df = tables[0]

    # Convert the "Date added" column to datetime
    sp500_df['Date added'] = pd.to_datetime(sp500_df['Date added'], errors='coerce')

    # Filter tickers added after the year 2000
    sp500_df = sp500_df[sp500_df['Date added'] > datetime(2000, 1, 1)]

    return sp500_df["Symbol"].tolist()

# Function to randomly select n tickers added after 2000
def select_random_tickers(n=10):
    tickers = get_sp500_tickers()
    if not tickers:
        return "Impossible to retrieve tickers."
    return random.sample(tickers, n)


# Function to get S&P 500 tickers
def get_sp500_tickers2():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url)
    sp500_df = tables[0]
    return sp500_df["Symbol"].tolist()

# Function to randomly select n tickers
def select_random_tickers2(n=10):
    tickers = get_sp500_tickers()
    if not tickers:
        return "Impossible to retrieve tickers."
    return random.sample(tickers, n)



# Function to download excess returns
def download_excess_returns(tickers, start_date, end_date):
    data = {}
    for ticker in tickers:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        if not stock_data.empty:
            price_column = 'Adj Close' if 'Adj Close' in stock_data.columns else 'Close'
            stock_data['Excess Returns'] = stock_data[price_column].pct_change() - risk_free_rate
            data[ticker] = stock_data['Excess Returns']
    return pd.DataFrame(data).dropna()  # Drop rows with missing values


# Function to optimize the source portfolio (phi_S)
def optimize_source_portfolio(mu_S, Sigma_S):
    def sharpe_ratio_source(phi, mu_S, Sigma_S):
        numerator = np.dot(mu_S, phi)
        denominator = np.sqrt(np.dot(phi.T, np.dot(Sigma_S, phi)))
        return -numerator / denominator  # Negative because we minimize

    result_source = minimize(sharpe_ratio_source, np.ones(len(mu_S)) / len(mu_S), args=(mu_S, Sigma_S),
                             method='SLSQP', bounds=[(0, 1) for _ in range(len(mu_S))],
                             constraints=[{'type': 'eq', 'fun': lambda phi: np.sum(phi) - 1}])
    return result_source.x

def optimize_source_portfolio2(mu_S, Sigma_S):
    def sharpe_ratio_source(phi, mu_S, Sigma_S):
        numerator = np.dot(mu_S, phi)
        denominator = np.sqrt(np.dot(phi.T, np.dot(Sigma_S, phi)))
        return -numerator / denominator  # Negative because we minimize

    result_source = minimize(sharpe_ratio_source, np.ones(len(mu_S)) / len(mu_S), args=(mu_S, Sigma_S),
                             method='SLSQP', bounds=[(0, 1) for _ in range(len(mu_S))],
                             constraints=[{'type': 'eq', 'fun': lambda phi: np.sum(phi) - 1}])
    return -result_source.fun

# Function to optimize the target portfolio with regularization (phi_T)
def optimize_target_portfolio(mu_T, Sigma_T, phi_S, lambda_reg):
    def regularized_sharpe_ratio(phi, mu_T, Sigma_T, phi_S, lambda_reg):
        numerator = np.dot(mu_T, phi)
        denominator = np.sqrt(np.dot(phi.T, np.dot(Sigma_T, phi)))
        sharpe_ratio = numerator / denominator
        regularization = lambda_reg * np.linalg.norm(phi_S - phi) ** 2  # Regularization term
        return -(sharpe_ratio - regularization)  # Negative because we minimize

    result_target = minimize(regularized_sharpe_ratio, np.ones(len(mu_T)) / len(mu_T),
                             args=(mu_T, Sigma_T, phi_S, lambda_reg), method='SLSQP',
                             bounds=[(0, 1) for _ in range(len(mu_T))],
                             constraints=[{'type': 'eq', 'fun': lambda phi: np.sum(phi) - 1}])
    return result_target.x, -result_target.fun

def optimize_target_portfolio2(mu_T, Sigma_T, phi_S, lambda_reg):
    def regularized_sharpe_ratio(phi, mu_T, Sigma_T, phi_S, lambda_reg):
        numerator = np.dot(mu_T, phi)
        denominator = np.sqrt(np.dot(phi.T, np.dot(Sigma_T, phi)))
        sharpe_ratio = numerator / denominator
        regularization = lambda_reg * np.linalg.norm(phi_S - phi) ** 2  # Regularization term
        return -(sharpe_ratio - regularization)  # Negative because we minimize

    result_target = minimize(regularized_sharpe_ratio, np.ones(len(mu_T)) / len(mu_T),
                             args=(mu_T, Sigma_T, phi_S, lambda_reg), method='SLSQP',
                             bounds=[(0, 1) for _ in range(len(mu_T))],
                             constraints=[{'type': 'eq', 'fun': lambda phi: np.sum(phi) - 1}])
    return -result_target.fun

# Function to compute transfer risk
# Function to compute R1 (Transfer Risk)
def compute_R1(phi, mu_T, Sigma_T):
    numerator = np.dot(mu_T, phi)
    denominator = np.sqrt(np.dot(phi.T, np.dot(Sigma_T, phi)))
    ratio = numerator / denominator
    R1 = 1.0 / ratio
    return R1

def compute_transfer_risk(phi_T, source_data, target_test_data):
    mu_S = np.array(source_data.mean())
    Sigma_S = np.array(source_data.cov())
    mu_T_test = np.array(target_test_data.mean())
    Sigma_T_test = np.array(target_test_data.cov())

    # Compute R1 (inverse of source portfolio Sharpe ratio)
    R1 = compute_R1(optimize_source_portfolio(mu_S, Sigma_S), mu_S, Sigma_S)

    # Compute R2 (Wasserstein-2 distance between source and target distributions)
    R2 = np.linalg.norm(mu_S - mu_T_test) + np.linalg.norm(Sigma_S - Sigma_T_test, ord=2)
    return R1 + R2



# Main experiment loop
n_experiments = 10  # Reduced to 100 for faster testing; increase to 1000 for final results
sharpe_ratios_transfer = []
sharpe_ratios_direct = []
transfer_risks = []

# Download target data
target_data = download_excess_returns(target_assets, target_start_date, test_end_date)
target_train_data = target_data.loc[:target_end_date]  # Training data up to February 2020
target_test_data = target_data.loc[test_start_date:]   # Testing data from February 2020 onwards

# Compute target mean and covariance for training data
mu_T = np.array(target_train_data.mean())
Sigma_T = np.array(target_train_data.cov())

for _ in range(n_experiments):
    try:
        # Randomly select source assets (10 random S&P 500 stocks)
        source_assets = select_random_tickers(n=10)
        print("Source Assets:", source_assets)

        # Generate source data
        source_data = download_excess_returns(source_assets, source_start_date, source_end_date)
        if not isinstance(source_data, pd.DataFrame) or source_data.shape[1] != 10:
            print("Skipping experiment due to missing data for some source assets.")
            continue

        # Optimize source portfolio (phi_S)
        mu_S = np.array(source_data.mean())
        Sigma_S = np.array(source_data.cov())
        phi_S = optimize_source_portfolio(mu_S, Sigma_S)

        # Optimize target portfolio with transfer learning (phi_T)
        phi_T, sharpe_ratio_transfer = optimize_target_portfolio(mu_T, Sigma_T, phi_S, lambda_reg)

        # Optimize target portfolio with direct learning (phi_T_direct)
        phi_T_direct = optimize_source_portfolio(mu_T, Sigma_T)

        # Compute target mean and covariance for testing data
        mu_T_test = np.array(target_test_data.mean())
        Sigma_T_test = np.array(target_test_data.cov())

        # Compute Sharpe ratios on testing data (inverser compute_R1 pour obtenir le vrai ratio)
        #sharpe_ratio_transfer_test = 1.0 / compute_R1(phi_S, mu_T_test, Sigma_T_test)
        #sharpe_ratio_direct_test = 1.0 / compute_R1(phi_T_direct, mu_T_test, Sigma_T_test)
        #phiTwhatever, sharpe_ratio_transfer_test = optimize_target_portfolio(mu_T_test, Sigma_T_test, phi_S, lambda_reg)
        sharpe_ratio_transfer_test = optimize_target_portfolio2(mu_T_test, Sigma_T_test, phi_S, lambda_reg)
        sharpe_ratio_direct_test= optimize_source_portfolio2(mu_T_test, Sigma_T_test)

        # Compute transfer risk
        R1 = compute_R1(phi_S, mu_S, Sigma_S)  # Correction : calculer R1 avec phi_S
        R2 = np.linalg.norm(mu_S - mu_T_test) + np.linalg.norm(Sigma_S - Sigma_T_test, ord=2)
        transfer_risk = R1 + R2

        # Store results
        sharpe_ratios_transfer.append(sharpe_ratio_transfer)
        sharpe_ratios_direct.append(sharpe_ratio_direct_test)
        transfer_risks.append(transfer_risk)

    except Exception as e:
        print(f"Error in experiment {_}: {e}")
        continue


# Compute correlation between Sharpe ratios and transfer risks
if len(sharpe_ratios_transfer) > 1 and len(transfer_risks) > 1:
    correlation, p_value = pearsonr(sharpe_ratios_transfer, transfer_risks)
    print("Correlation between Sharpe Ratio (Transfer) and Transfer Risk:", correlation)
    print("p-value:", p_value)
else:
    print("Not enough experiments completed to compute correlation.")
    print(transfer_risks)
    print(sharpe_ratios_transfer)

    print(sharpe_ratios_direct)